In [1]:
import numpy as np
import pandas as pd

from scipy import stats
from statistics import median

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

SEED = 1

2022-05-14 01:42:39.845609: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-14 01:42:39.845671: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = pd.read_csv('AMLtrain.csv')

In [3]:
target_col = ['respsimple']
y = data[target_col[0]]

#change output to 0, 1 class label
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

In [4]:
cat_columns = ['Patient_id','SEX','PRIORMAL','PRIORCHEMO','PRIORXRT','Infection','cytocat','ITD','D835','RasStat','ChemoSimplest'
]

# remove categorical columns
X = data.copy().drop(columns=cat_columns).drop(columns=target_col)

# use for mean/median/mode
for col in X.columns:
#     mean
    X[col] = X[col].fillna(X[col].mean())
    
#     mode
#     m = stats.mode(X[col])[0][0]
#     if np.isnan(m):
#         X[col] = X[col].fillna(X[col].mean())
#     else:
#         X[col] = X[col].fillna(m)

#     median
#     m = median(X[col])
#     if np.isnan(m):
#         X[col] = X[col].fillna(X[col].mean())
#     else:
#         X[col] = X[col].fillna(m)
    
# kNN
# imputer = KNNImputer(n_neighbors=2)
# df_col = X.columns
# X = pd.DataFrame(imputer.fit_transform(X))
# X.columns = df_col

X.head()

,AgeatDx,AHD,WBC,ABSBLST,BMBLAST,BMMONOCYTES,BMPROM,PBBLAST,PBMONO,PBPROM,...,VHL,WTAP,XIAP,XPO1,YAP1,YAP1p,YWHAE,YWHAZ,ZNF296,ZNF346
0,63.85,0,10.8,1188.0,35.0,36.000000,2.000000,11.0,17.000000,0.000000,...,-0.722347,-0.887311,1.140104,-0.199574,-0.598099,-1.086189,-1.182815,0.336552,-0.291540,-0.156785
1,57.45,24,2.7,0.0,26.0,0.000000,0.000000,0.0,0.500000,0.000000,...,-0.401240,0.228719,2.025590,-0.268515,0.035639,-0.498447,-0.391924,-0.654588,0.110702,1.260718
2,59.49,5,1.3,0.0,35.0,1.000000,2.000000,0.0,5.000000,0.000000,...,0.322115,0.578631,-1.918884,-1.485057,-0.214276,-0.115862,0.503870,-0.263103,-0.500761,0.147560
3,46.38,0,0.8,16.0,0.0,4.162011,1.413408,2.0,11.900463,0.608333,...,0.460794,-0.724971,2.274829,0.837169,-0.240740,-0.788800,-0.986515,-0.510208,0.910813,-0.303119
4,66.95,0,2.0,160.0,34.0,3.000000,1.000000,8.0,5.000000,0.000000,...,-0.330346,-1.460989,0.004068,-1.152868,0.450986,0.753539,0.011574,-0.973533,-0.599299,0.459281


In [5]:
# split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [6]:
# normalize data values
norm = MinMaxScaler().fit(X_train)

X_train = norm.transform(X_train)
X_test = norm.transform(X_test)

In [30]:
# train model
n_epochs = 100
batch_size = 10

callbacks = [EarlyStopping(monitor='val_accuracy', patience=20)]

kf = KFold(n_splits=5, shuffle=False, random_state=None)

cv_val_loss = []
cv_val_acc= []
for train_idx, test_idx in kf.split(X_train):
    model = Sequential()
    model.add(Dense(256, input_dim=len(X.columns), activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    
    
    history = model.fit(X_train[train_idx], y_train[train_idx], epochs=n_epochs, batch_size=batch_size, validation_split=0.1, callbacks=callbacks)
    score = model.evaluate(X_train[test_idx], y_train[test_idx])
    cv_val_loss.append(score[0])
    cv_val_acc.append(score[1])
    
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('model loss')
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()
# model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.1, callbacks=callbacks)

Epoch 1/100
11/11 [==============================] - 0s 6ms/step - loss: 0.6336 - accuracy: 0.7379 - val_loss: 0.6823 - val_accuracy: 0.5833
Epoch 2/100
11/11 [==============================] - 0s 2ms/step - loss: 0.6006 - accuracy: 0.7476 - val_loss: 0.6974 - val_accuracy: 0.5833
Epoch 3/100
11/11 [==============================] - 0s 2ms/step - loss: 0.5665 - accuracy: 0.7379 - val_loss: 0.7347 - val_accuracy: 0.5833
Epoch 4/100
11/11 [==============================] - 0s 2ms/step - loss: 0.5385 - accuracy: 0.7476 - val_loss: 0.7257 - val_accuracy: 0.5833
Epoch 5/100
11/11 [==============================] - 0s 2ms/step - loss: 0.5212 - accuracy: 0.7476 - val_loss: 0.7479 - val_accuracy: 0.5833
Epoch 6/100
11/11 [==============================] - 0s 2ms/step - loss: 0.5216 - accuracy: 0.7476 - val_loss: 0.6728 - val_accuracy: 0.5833
Epoch 7/100
11/11 [==============================] - 0s 2ms/step - loss: 0.5069 - accuracy: 0.7379 - val_loss: 0.7290 - val_accuracy: 0.5833
Epoch 8/100
1

Epoch 16/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3599 - accuracy: 0.8544 - val_loss: 0.6809 - val_accuracy: 0.5833
Epoch 17/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3491 - accuracy: 0.8447 - val_loss: 0.6308 - val_accuracy: 0.5833
Epoch 18/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3321 - accuracy: 0.8544 - val_loss: 0.6285 - val_accuracy: 0.5833
Epoch 19/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3223 - accuracy: 0.9126 - val_loss: 0.6984 - val_accuracy: 0.5833
Epoch 20/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3112 - accuracy: 0.8932 - val_loss: 0.6987 - val_accuracy: 0.5833
Epoch 21/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3001 - accuracy: 0.8641 - val_loss: 0.5938 - val_accuracy: 0.6667
Epoch 22/100
11/11 [==============================] - 0s 2ms/step - loss: 0.2986 - accuracy: 0.9223 - val_loss: 0.6339 - val_accuracy: 0.5833
Epoch 

11/11 [==============================] - 0s 2ms/step - loss: 0.3625 - accuracy: 0.8462 - val_loss: 0.6629 - val_accuracy: 0.7500
Epoch 16/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3573 - accuracy: 0.9038 - val_loss: 0.6226 - val_accuracy: 0.8333
Epoch 17/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3420 - accuracy: 0.8942 - val_loss: 0.6308 - val_accuracy: 0.7500
Epoch 18/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3371 - accuracy: 0.8365 - val_loss: 0.7454 - val_accuracy: 0.6667
Epoch 19/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3258 - accuracy: 0.9327 - val_loss: 0.7282 - val_accuracy: 0.8333
Epoch 20/100
11/11 [==============================] - 0s 2ms/step - loss: 0.3600 - accuracy: 0.8654 - val_loss: 0.9182 - val_accuracy: 0.3333
Epoch 21/100
1/1 [==============================] - 0s 10ms/step - loss: 0.5621 - accuracy: 0.7143


In [31]:
print(f"CV Error: {np.mean(cv_val_loss)}")
print(f"CV Error (All): {cv_val_loss}")
print(f"CV Accuracy: {np.mean(cv_val_acc)}")
print(f"CV Accuracy (All): {cv_val_acc}")

CV Error: 0.5944384694099426
CV Error (All): [0.509949803352356, 0.7067073583602905, 0.5643715858459473, 0.6290499567985535, 0.5621136426925659]
CV Accuracy: 0.7428571462631226
CV Accuracy (All): [0.7586206793785095, 0.6896551847457886, 0.7586206793785095, 0.7931034564971924, 0.7142857313156128]


In [32]:
score = model.evaluate(X_test, y_test)

print(f"Test Loss: {score[0]}")
print(f"Test Accuracy: {score[1]}")

2/2 [==============================] - 0s 1ms/step - loss: 0.5292 - accuracy: 0.7568
Test Loss: 0.5291962027549744
Test Accuracy: 0.7567567825317383


In [41]:
model.save('respsimple')

INFO:tensorflow:Assets written to: respsimple/assets
